In [2]:
import numpy as np
from nltk.corpus import brown, stopwords
from nltk.cluster.util import cosine_distance
import tensorflow as tf
import pandas as pd
from collections import defaultdict
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize, tokenize
from nltk.stem import WordNetLemmatizer 
import io
from operator import itemgetter 

In [ ]:
meta_data = []
meta_path= '/Users/alexhe/Desktop/cs221project/meta_Books.json'
with open(path) as f:
    for line in f:
        meta_data.append(json.loads(line))

asin_title_map = defaultdict(list)

for i in meta_data:
    if 'title' in i.keys():
        asin_title_map[i['asin']] = i['title']
    else:
        asin_title_map[i['asin']] = None

In [3]:
def sentence_similarity(s1, s2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in s1.split()]
    sent2 = [w.lower() for w in s2.split()]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)

In [4]:

# One out of 5 words differ => 0.8 similarity
print(sentence_similarity("This is a good sentence", "This is a bad sentence"))

stop_words = stopwords.words('english')

0.7999999999999998


In [5]:

def build_similarity_matrix(sentences, stopwords=None):
    # Create an empty similarity matrix
    S = np.zeros((len(sentences), len(sentences)))
 
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2:
                continue
       
            S[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
 
    # normalize the matrix row-wise
    for idx in range(len(S)):
        if S[idx].sum() == 0:
            continue
        else:
            S[idx] /= S[idx].sum()
 
    return S

In [6]:
def pagerank(A, eps=0.0001, d=0.85):
    P = np.ones(len(A)) / len(A)
    while True:
        new_P = np.ones(len(A)) * (1 - d) / len(A) + d * A.T.dot(P)
        delta = abs(new_P - P).sum()
        if delta <= eps:
            return new_P
        P = new_P


In [7]:
def textrank(sentences, top_n=5, stopwords=None):
    """
    sentences = a list of sentences [[w11, w12, ...], [w21, w22, ...], ...]
    top_n = how may sentences the summary should contain
    stopwords = a list of stopwords
    """
    S = build_similarity_matrix(sentences, stop_words) 

    sentence_ranks = pagerank(S)
 
    # Sort the sentence ranks
    ranked_sentence_indexes = [item[0] for item in sorted(enumerate(sentence_ranks), key=lambda item: -item[1])]
    selected_sentences = sorted(ranked_sentence_indexes[:top_n])
    summary = itemgetter(*selected_sentences)(sentences)
    return summary
 

In [8]:
path = '/Users/alexhe/Desktop/cs221project/reviews_Books_5.json'
data = []
counter = 0
num = 20000
with open(path) as f:
    for line in f:
        if counter < num:
            counter += 1
            data.append(json.loads(line))
        else: 
            break


In [9]:

X=[] #reviews
Y=[] #ratings
asin_dict = defaultdict(list)
review_count = 0

# Assigns X as a list of reviews, Y as a list of star ratings, and asin_dict as a dictionary with keys asin_pos and asin_neg
# where asin_pos contains all reviews associated with that asin with a star-rating of 4 or 5, and asin_neg has all reviews
# for that asin with a star rating of 1, 2, or 3
for i in data:
    review_count += 1
    X.append(i['reviewText'])
    Y.append(i['overall'])
    if i['overall'] >= 4:
        asin_dict[i['asin']+"_pos"].append(i['reviewText'])
    else:
        asin_dict[i['asin']+"_neg"].append(i['reviewText'])

# Merges the list of reviews for each asin_pos and asin_neg into a single string
for j in asin_dict.keys():
    asin_dict[j] = " ".join(asin_dict[j])

In [10]:
print review_count
print X[1]
print Y[1]


20000
This is one my must have books. It is a masterpiece of spirituality. I'll be the first to admit, its literary quality isn't much. It is rather simplistically written, but the message behind it is so powerful that you have to read it. It will take you to enlightenment.
5.0


In [11]:
dict = asin_dict.keys()
print dict[0]
print asin_dict[dict[0]]

000100039X_pos
Spiritually and mentally inspiring! A book that allows you to question your morals and will help you discover who you really are! This is one my must have books. It is a masterpiece of spirituality. I'll be the first to admit, its literary quality isn't much. It is rather simplistically written, but the message behind it is so powerful that you have to read it. It will take you to enlightenment. This book provides a reflection that you can apply to your own life.And, a way for you to try and assess whether you are truly doing the right thing and making the most of your short time on this plane. I first read THE PROPHET in college back in the 60's. The book had a revival as did anything metaphysical in the turbulent 60's. It had a profound effect on me and became a book I always took with me. After graduation I joined the Peace Corps and during stressful training in country (Liberia) at times of illness and the night before I left, this book gave me great comfort. I read 

In [12]:
for key in asin_dict.keys():
    asin_dict[key] = tokenize.sent_tokenize(asin_dict[key])


In [27]:
holder = True
i = 0
count = 0

while holder:

    if len(asin_dict[dict[i]]) >= 100 and len(asin_dict[dict[i]]) <=200:
        count += 1
        print dict[i]
        print len(asin_dict[dict[i]])
        #print asin_dict[dict[i]]
        for idx, sentence in enumerate(textrank(asin_dict[dict[i]], stopwords=stop_words)):
            print("%s. %s" % ((idx + 1), ' '.join(sentence)))
    if count == 5:
        break
    i += 1

0006470238_pos
139
1. T h a t   i n   i t s e l f   m a k e s   t h e   G a p   S e r i e s   q u i t e   u n i q u e   i n   a n y   b o o k s   i ' v e   r e a d   b e f o r e . I t   w a s   a   r e a l   h a r d   s l o g   t o   g e t   t h r o u g h   a l l   o f   t h e s e   b o o k s   a n d   t o   b e   h o n e s t   I   w o n t   b e   r e p e a t i n g   t h e   e x p e r i e n c e   a n y   t i m e   s o o n .
2. A s   t h e   b o o k s   w e n t   o n ,   t h e   c h a r a c t e r s   g a i n e d   d e p t h ,   t h e   p l o t   t w i s t e d   a n d   g r e w   m o r e   p r e c a r i o u s ,   a n d   I   s t a r t e d   t a k i n g   s i d e s . B y   t h e   t i m e   I   r e a c h e d   t h i s   f i n a l   b o o k   o f   t h e   s a g a ,   I   w a s   f u l l y   h o o k e d ,   a n d   r i g h t l y   s o .
3. T h i s   i s   a s   i t   s h o u l d   b e   -   a   s e r i e s   a b o u t   a   p e r i o d   i n   t i m e ,   n o t   a   n o v e l   c l a i m 